In [ ]:
import numpy as np

In [ ]:
### Code is based on the second reference of the assignment's paper

#### USEFUL REFERENCES #######
## Initialisations, FV, VF according to https://www.ics.uci.edu/~welling/teaching/ICS279/LPCD.pdf
## FV, VF p 7, 8 --- Initialisation p 8
## parity check function: https://en.wikipedia.org/wiki/Hamming_code

In [ ]:
H=np.loadtxt('H1.txt')
y=np.loadtxt('y1.txt')

In [ ]:
print(H.shape)
print(y.shape)

In [ ]:
## find the indexes of the factors with value one for a specific variable
## that indicates the existance of an edge between the variable and the factor at the bipartite graph
def get_active_edges(node):   
    result = []
    for i in range(node.shape[0]):
        if(node[i] == 1):
            result.append(i)
    return result

In [ ]:
def create_var_edge_dict(H):
    variable_number = H.shape[1]
    var_edge_dict = {}
    for i in range(variable_number):
        var_edge_dict[i] = get_active_edges(H[:,i])
        
    return var_edge_dict

In [ ]:
def create_factor_edge_dict(H):
    factor_number = H.shape[0]
    factor_edge_dict = {}
    for i in range(factor_number):
        factor_edge_dict[i] = get_active_edges(H[i,:])
        
    return factor_edge_dict

In [ ]:
def Factor_to_Variable_message(factor_edge_dict,H,Message):

    FV_Message = np.zeros((H.shape))
    
    ## passing all the factor to variable messages
    
    factors_number = H.shape[0]
    variables_number = H.shape[1]
    
    ## for every factor
    for factor_id in range(factors_number):
        
        ## find the active factor to variable edges
        active_edges = factor_edge_dict.get(factor_id)
        
        ## for every active factor
        for active_factor_index_1 in active_edges:
            message_product = 1
            ## for every other active factor
            for active_factor_index_2 in active_edges:
                ## that is no the same with the first one
                if(active_factor_index_1 != active_factor_index_2):
                    message_product = message_product * np.tanh(Message[factor_id,:][active_factor_index_2]/2)
                    
            FV_Message[factor_id,active_factor_index_1]=np.log((1+message_product)/(1-message_product))
            
    return FV_Message

In [ ]:
def Variable_to_Factor_message(var_edge_dict,Ditribution,Message,FV_message,H,iteration):
    
    Message = np.zeros((H.shape))
    
    ## compute the initial var to factor message matrix, passing messages from y to H
    ## only for the first iteration 
    if(iteration == 0):    
        for i in range(H.shape[0]):
            for j in range(H.shape[1]):
                Message[i][j] = H[i][j]*Ditribution[j]
                
        return Message
                
        
    else: ## update the already obtained message matric computing the variable to factor messages
        
        Message = np.array(Message)
        
        ## take all the incoming messages apart from the one we are going to send the message to 
        factor_number,variable_number = FV_message.shape
    
        for variable_index in range(variable_number):
        
            ## get the active edges of each variable
            active_edges = var_edge_dict.get(variable_index)
        
            for edge_index_1 in active_edges:
                message = 0
                for edge_index_2 in active_edges:
                    if(edge_index_1!=edge_index_2):
                        message = FV_message[:,variable_index][edge_index_2]+message
                
                Message[edge_index_1,variable_index] = Ditribution[variable_index] + message
            
        return Message
        

In [ ]:
## compute the marginal distribution based on the last factor to variable message
def update_message(Ditribution,FV_Message):
    
    marginal_message = []
    
    variables_number = FV_Message.shape[1]
    
    ## for every variable sum all the incoming factor to variable messages
    for i in range(variables_number):
        ## sum over columns --> variables are denoted with columns
        marginal_message.append(Ditribution[i] + np.sum(FV_Message,axis = 0)[i])
        
    return marginal_message
    

In [ ]:
## compute the result of a round based on the last variable to factor message computed
## VF message contains real numbers --> convert them into 0,1
def compute_result(VF_message):
    result = []
    for bit in VF_message:
        if(bit > 0):
            result.append(1)
        else:
            result.append(0)
    return result

In [ ]:
def BinarySymmetricChannel_Initialisation(y, H, p):
    
    distribution = []
    
    ## for thr Binary Symmetric Channel the probability log(P(y|x)) = (x-y)(x-y+1)log(p/(1-p))
    for i in range(len(y)):
        if(y[i]==1):
            distribution.append(np.log((1-p)/p))
        else:
            distribution.append(np.log(p/(1-p)))

    return np.array(distribution)

In [ ]:
## check how many erros occur in the decoding we made
def parity_check(z):
    res = z%2
    wrong_bits = np.count_nonzero(res)
    if(wrong_bits == 0):
        return True,wrong_bits
    else:
        return False,wrong_bits  

In [ ]:
def LDPC_Decoder(y, H, p=0.1, max_iter=20):
    
    ## creating to dictionaries
    ## defining the active edges of the bipartite graph
    ## factor ---> variable edges
    ## variable ----> factor edges
    var_edge_dict = create_var_edge_dict(H)
    factor_edge_dict = create_factor_edge_dict(H)
    
    
    # y: the received codeword matrix
    # H: LDPC Matrix
    # p: noise ratio
    # max_iter: maximum number of iterations of loopy belief propagation algorithm
    # Ditribution: p(y|x_n = 0,1) matrix
    # FV_message: Factor to Variable message matrix
    # VF_message: Variable to Factor message matrix
    # Message: Marginal Distribution Message Matrix
    
    Ditribution = BinarySymmetricChannel_Initialisation(y,H,p)
    FV_message = None
    Message = None
    
    ##### PROCEDURE ####################################################################################
    ## 1) Variable to factor Messages
    ## !!! for the first round the VF message is computed in a special way -- look the VF function
    ## 2) Factor to Variable Messages
    ## 3) Compute the marginal distribution
    ## 4) Check our message
    ## 5) Go to 1 and loop over again
    #####################################################################################################
    
    
    for i in range(max_iter):
        print("Iteration " + str(i + 1))
        
        ## compute variable to factor message
        VF_message = Variable_to_Factor_message(var_edge_dict,Ditribution,Message,FV_message,H,i)
        ## compute factor to variable message
        FV_message = Factor_to_Variable_message(factor_edge_dict,H,VF_message)
        ## compute the marginal distribution of the round
        Message = update_message(Ditribution,FV_message)
        ## compute the result after the specific iterartion -- binary format
        result_of_round = compute_result(Message)
        
        ## a check needed here in order to determine if have computed a valid codeword
        z = H @ result_of_round # z is the syndrome
        
        ## compute th syndrome in order to detect whether is a wrong bit in the codeword
        term_flag,wrong_bits = parity_check(z)
        if(term_flag):
            print("Decoded succesfully!")
            return result_of_round
        else:
            print("Decoding failure with " + str(wrong_bits) + " wrong bits")
            if(i == max_iter - 1):
                return -1
        
        

In [ ]:
res = LDPC_Decoder(y,H,p=0.3,max_iter = 90)
if(res == -1):
    print("Failure in decoding!")

In [ ]:
def recover_message(message):
    
    ### smash the recover message in words of 8 bits each
    ### each word represents a character
    ### result is a list of lists, each inner list represents a word/character
    flag = True
    start_index = 0
    result = []
    while(flag):
        ## append every word
        result.append(message[start_index:start_index + 8])
        start_index += 8
        if(start_index > 248):
            flag = False
            
            
    ## convert each word in the corresponding decimal
    dec_result = []
    for word in result:
        w = ''
        for bit in word:
            w += str(int(bit))
        dec_result.append(int(w,2))
        
    ## convert each decimal to a character based on its ASCII
    ## finally concatenate them in order to get the final message
    message = ''
    for word in dec_result:
        message += chr(word)
        
    return(message)

In [ ]:
print(str(recover_message(res)))